In [1]:
import pandas as pd
from arango import ArangoClient

In [78]:
raw_df = pd.read_csv('./data/raw.csv')

In [2]:
client = ArangoClient(hosts="https://2ae4f052d710.arangodb.cloud:8529")

db = client.db('machine_learning', username='lab_test', password='z-rRdN-Enf4qQwybGiVdbG')

In [143]:
# aql_query = """
#     FOR org IN organizations
#         LET outboundPaths = (
#             FOR path IN SHORTEST_PATH org._id TO transactions
#                 RETURN LENGTH(path.vertices) - 1
#         )
#         LET inboundPaths = (
#             FOR path IN SHORTEST_PATH org._id FROM transactions
#                 RETURN LENGTH(path.vertices) - 1
#         )
#         LET outboundCentrality = COUNT(outboundPaths) > 0 ? 1 / AVERAGE(outboundPaths) : 0
#         LET inboundCentrality = COUNT(inboundPaths) > 0 ? 1 / AVERAGE(inboundPaths) : 0
#         LET combinedCentrality = (outboundCentrality + inboundCentrality) / 2
#         RETURN {
#             organization: org,
#             outboundCentrality: outboundCentrality,
#             inboundCentrality: inboundCentrality,
#             combinedCentrality: combinedCentrality
#         }
# """

# aql_query = """
#     FOR org in organizations
#         LET outboundPaths = (
#             LENGTH(TRAVERSAL(org._id, 'transactions', {direction: 'outbound'}))
#         )
#     RETURN { org: outboundPaths }
# """

# aql_query = """
#     FOR org in organizations
        
# """


aql_query = """
FOR org IN organizations
    LIMIT 1
    FOR other_org IN organizations
        FILTER org != other_org
        LET inbound_path = (
            FOR vertex IN INBOUND SHORTEST_PATH org TO other_org 
            GRAPH organization_organization_transaction
            RETURN vertex
        )
        LET outbound_path = (
            FOR vertex IN OUTBOUND SHORTEST_PATH org TO other_org 
            GRAPH organization_organization_transaction
            RETURN vertex
        )
        LET outbound_path_length = LENGTH(outbound_path)
        LET inbound_path_length = LENGTH(inbound_path)
        FILTER (inbound_path_length > 0) OR (outbound_path_length > 0)
        RETURN {
            org: org._id,
            other_org: other_org._id,
            outbound_path_length: (outbound_path_length > 0 ? outbound_path_length - 1 : 0),
            inbound_path_length: (inbound_path_length > 0 ? inbound_path_length - 1 : 0)
        }
"""

# aql_query = """
# FOR org IN organizations
#     LIMIT 1
#     FOR other_org IN organizations
#         FILTER org != other_org
#         LET path = (
#             FOR vertex IN OUTBOUND SHORTEST_PATH org TO other_org 
#             GRAPH organization_organization_transaction
#         )
#         LET path_length = LENGTH(path)
#         RETURN { 
#             org: org._id, 
#             other_org: other_org._id, 
#             path_length: path_length 
#         }
# """

# aql_query = """
# FOR v IN organizations
#     LET outbound_degree_centrality = LENGTH(
#         FOR e IN transactions
#             FILTER e._from == v._id
#             RETURN e
#     )
#     LET inbound_degree_centrality = LENGTH(
#         FOR e IN transactions
#             FILTER e._to == v._id
#             RETURN e
#     )
#     LET total_degree_centrality = outbound_degree_centrality + inbound_degree_centrality

#     RETURN {
#         organizations: v._id,
#         inbound_degree_centrality: inbound_degree_centrality,
#         outbound_degree_centrality: outbound_degree_centrality,
#         total_degree_centrality: total_degree_centrality
#     }
# """

In [144]:
cursor = db.aql.execute(aql_query)

In [145]:
for doc in cursor:
    print(doc)

{'org': 'organizations/4a44d0b141bfe5a280c158bd8fbb56f5', 'other_org': 'organizations/6eeec4b8f6d2ddcfac4d0f6c13f60de9', 'outbound_path_length': 2, 'inbound_path_length': 0}
{'org': 'organizations/4a44d0b141bfe5a280c158bd8fbb56f5', 'other_org': 'organizations/8cb8fdfdf76f13813699a403a2f70974', 'outbound_path_length': 1, 'inbound_path_length': 0}
{'org': 'organizations/4a44d0b141bfe5a280c158bd8fbb56f5', 'other_org': 'organizations/336ede5aa5fa1d0ab5c6afebad427598', 'outbound_path_length': 2, 'inbound_path_length': 0}


In [118]:
raw_df[raw_df.sender_id == '4a44d0b141bfe5a280c158bd8fbb56f5']

,country_of_destination,country_of_origin,hs6_code,receiver_address,receiver_id,receiver_name,sender_address,sender_id,sender_name,tier,transaction_date
60883,IN,NaN,292219,"204, SIDDHARTH COMPLEXR C DUTT ROAD, ALKAPURI ...",8cb8fdfdf76f13813699a403a2f70974,Admark Polycoats Private Limited,Germany,4a44d0b141bfe5a280c158bd8fbb56f5,VINK CO GMBH,t2->t1,3/19/22
60884,IN,NaN,292219,"204, SIDDHARTH COMPLEXR C DUTT ROAD, ALKAPURI ...",8cb8fdfdf76f13813699a403a2f70974,Admark Polycoats Private Limited,Germany,4a44d0b141bfe5a280c158bd8fbb56f5,VINK CO GMBH,t2->t1,4/5/22
60885,IN,NaN,292219,"204, SIDDHARTH COMPLEXR C DUTT ROAD, ALKAPURI ...",8cb8fdfdf76f13813699a403a2f70974,Admark Polycoats Private Limited,Germany,4a44d0b141bfe5a280c158bd8fbb56f5,VINK CO GMBH,t2->t1,3/19/22
60886,IN,NaN,292219,"204, SIDDHARTH COMPLEXR C DUTT ROAD, ALKAPURI ...",8cb8fdfdf76f13813699a403a2f70974,Admark Polycoats Private Limited,Germany,4a44d0b141bfe5a280c158bd8fbb56f5,VINK CO GMBH,t2->t1,3/10/22
60887,IN,NaN,292219,"204, SIDDHARTH COMPLEXR C DUTT ROAD, ALKAPURI ...",8cb8fdfdf76f13813699a403a2f70974,Admark Polycoats Private Limited,Germany,4a44d0b141bfe5a280c158bd8fbb56f5,VINK CO GMBH,t2->t1,12/15/21


In [129]:
raw_df[raw_df.receiver_id == '4a44d0b141bfe5a280c158bd8fbb56f5']

,country_of_destination,country_of_origin,hs6_code,receiver_address,receiver_id,receiver_name,sender_address,sender_id,sender_name,tier,transaction_date


In [ ]:
raw_df